In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()
all_data = project.get_all_data()

# Rock Typing

The process of rock typing is a multidisciplinary effort that involves close collaboration between geologists, petrophysicists, and reservoir engineers. Each discipline plays a specific role, and the rock typing workflow involves a seamless transition of data, models, and insights across these domains to build a robust understanding of the reservoir. Below is the high-level workflow detailing the steps, key tasks, and handoffs between these professionals

Petrophysicists take geological inputs and combine them with well log data and core analysis to define rock types based on petrophysical properties such as porosity, permeability, fluid saturation, and pore geometry.

Tasks:
- Core Data Integration:
    - Perform routine and special core analyses (RCAL, SCAL) to measure porosity, permeability, capillary pressure, and wettability.

- Log Interpretation:
    - Interpret wireline logs (e.g., density, neutron, resistivity, NMR) to generate continuous petrophysical properties along the wellbore.

- Rock Typing Methods:
    - Use statistical tools like Flow Zone Indicator (FZI), Pickett plots, and cluster analysis to group rocks with similar flow behavior into petrophysical rock types.

- Electrofacies Analysis:
    - Identify electrofacies by clustering log responses, especially where core data is limited.

The rock typing workflow is a collaborative process where geologists, petrophysicists, and reservoir engineers work together to ensure the reservoir is accurately characterized and modeled. This interdisciplinary approach ensures that both geological and petrophysical complexities are accounted for, resulting in more efficient production strategies and better field development plans

## Identifying the number of rock types

Different methods have been discussed by previous works on determining of the number of rock types for a given core data. Among others are Wards Plot, Modified Lorenz Plot and its extensions.

1. Flow Zone Index (FZI) Method
    - The FZI method classifies rock types based on their flow characteristics. It involves calculating the Flow Zone Index from core data and using it to group similar rock types. This method is particularly effective in heterogeneous reservoirs3.

1. Ward’s Method
    - Ward’s method is a hierarchical clustering technique that minimizes the total within-cluster variance. It starts with each observation in its own cluster and merges clusters iteratively to minimize the increase in total within-cluster variance1. This method is particularly useful for quantitative variables and can be visualized using a dendrogram, which helps in identifying the optimal number of clusters (rock types).

2. Lorenz Method
    - The Lorenz method, often used in petrophysical analysis, involves plotting cumulative storage capacity against cumulative flow capacity. This method helps in identifying distinct rock types based on their flow properties. By analyzing the Lorenz plot, you can determine the number of rock types and their respective contributions to storage and flow capacities2.

However, since the lithofacies data is not available to calibrate the rock typing, Costa resolved to defining the rock types based on Winland R35 method and resulted in 27 RRTs.

For simplicity, this work break the rocks into 4 rock types only based on the Pore Throat Size (PTS) classification below;
Rock Type 1: Mega: PTS > 10 microns
Rock Type 2: Macro: PTS > 2 microns
Rock Type 3: Meso: PTS > 0.5 microns
Rock Type 4: Micro: PTS > 0.1 microns

In [ ]:
import pandas as pd
import numpy as np
from quick_pp.rock_type import plot_ward, plot_modified_lorenz

core_data = pd.read_csv(r'data\01_raw\HW_core_data_all.csv')
core_data['CPORE'] = core_data['Phi (frac.)']
core_data['CPERM'] = core_data['K mD']
core_data['PC'] = core_data['O/B Pc (psia)']
core_data['SW'] = core_data['Equiv Brine Sat. (Frac.)']
core_data = core_data[core_data['CPERM'] > 0]

clean_core_data = core_data.drop_duplicates(subset=['Sample', 'CPORE', 'CPERM'], keep='last')
plot_ward(clean_core_data['CPORE'], clean_core_data['CPERM'])
plot_modified_lorenz(clean_core_data['CPORE'], clean_core_data['CPERM'])

log_fzi_cut_offs = [-.679, -.179, .171, .571, .871]
fzi_cut_offs = [round(10**(i), 3) for i in log_fzi_cut_offs]
print(fzi_cut_offs)

xxx explain the core data xxx

Based on the Ward's plot above, it is deduced that the data can be grouped into 4 rock types where the limits of log(FZI) values are -0.679, -0.179, 1.71, 0.571 and 0.871.
This translates into FZI values of 0.209, 0.662, 1.483, 3.724, 7.43. Consequently, the rock types are categorized as follows;
- Rock Type 1: FZI >= 3.724
- Rock Type 2: 1.483 <= FZI < 3.724
- Rock Type 3: 0.662 <= FZI < 1.483
- Rock Type 4: FZI < 0.662

## Plotting the Rock Types

In [ ]:
from quick_pp.rock_type import plot_fzi, plot_rfn, plot_winland, calc_fzi, rock_typing
from quick_pp.rock_type import calc_r35, rock_typing, plot_winland

all_data['CPERM'] = all_data.CORE_PERM
all_data['CPORE'] = all_data.CORE_POR / 100
# all_data['CPERM'] = np.where(all_data.CPERM < .01, np.nan, all_data.CPERM)
# all_data['CPORE'] = np.where(all_data.CPORE < .01, np.nan, all_data.CPORE)

# Estimate rock types
r35 = calc_r35(all_data['CPORE'], all_data['CPERM'])
all_data['R35'] = r35
r35_cut_offs = [.1, .5, 2, 10, 100]
rock_flag = rock_typing(r35, higher_is_better=True, cut_offs=r35_cut_offs)
all_data['ROCK_FLAG'] = rock_flag

print('The data has been classified into the following rock types:')
print(all_data['ROCK_FLAG'].value_counts())

As a comparison, the rock types being identified using FZI is plotted on Winland R35 and Lucia Rock Fabric Number (RFN) methods.

- The plot demonstrates Winland R35 resulted in a more flat permeability estimation while
- Lower porosity - high permeability datapoints.
    - These points might be indicating fracture kind of rock types
    - Both Winland R35 and Lucia RFN does not model the datapoints
    - FZI is able to model but indicates too high of a value

In [ ]:
plot_fzi(all_data['CPORE'], all_data['CPERM'], rock_type=rock_flag)
plot_winland(all_data['CPORE'], all_data['CPERM'], rock_type=rock_flag)
plot_rfn(all_data['CPORE'], all_data['CPERM'], rock_type=rock_flag)

Determining the perm transform parameter for each rock type.

In [ ]:
from ipywidgets import interact, widgets
from quick_pp.core_calibration import poroperm_xplot

rt = widgets.Dropdown(
    options=all_data['ROCK_FLAG'].dropna().unique(),
    value=1,
    description='Rock Type:'
)
a = widgets.FloatSlider(
    value=1000,
    min=.1,
    max=10000,
    step=1,
    readout_format='d'
)
b = widgets.FloatSlider(
    value=2,
    min=.1,
    max=3,
    step=.1,
    readout_format='.1f'
)

@interact(rt=rt, a=a, b=b)
def param(rt, a, b):
    data = all_data[all_data['ROCK_FLAG'] == rt]
    poroperm_xplot(data['CPORE'], data['CPERM'], a=a, b=b, log_log=False)

### Compare different PERM estimations

In [ ]:
import pickle
from quick_pp.rock_type import train_rock_type, train_fzi

train_data = all_data.copy()  # Filter out low porosity data
# train_data = train_data[train_data.CPORE >= .02].copy()  # Filter out low porosity data
# train_data = train_data[train_data.CPERM > .1]  # Filter out low permeability data
train_data = train_data[train_data.ROCK_FLAG < 5]  # Filter out non-reservoir data

train_data = train_data.dropna(subset=['GR', 'NPHI', 'RHOB', 'ROCK_FLAG'])
rt_model = train_rock_type(train_data[['GR', 'NPHI', 'RHOB']], train_data['ROCK_FLAG'])
with open(r'data\04_project\MOCK_carbonate\outputs\rt_model.qppm', 'wb') as file:
    pickle.dump(rt_model, file)

# train_data = train_data.dropna(subset=['GR', 'NPHI', 'RHOB', 'FZI'])
# fzi_model = train_fzi(train_data[['GR', 'NPHI', 'RHOB']], train_data['FZI'], train_data['ROCK_FLAG'])
# with open(r'data\04_project\MOCK_carbonate\outputs\fzi_model.qppm', 'wb') as file:
#     pickle.dump(fzi_model, file)

train_data = train_data.dropna(subset=['GR', 'NPHI', 'RHOB', 'R35'])
r35_model = train_fzi(train_data[['GR', 'NPHI', 'RHOB']], train_data['R35'], train_data['ROCK_FLAG'])
with open(r'data\04_project\MOCK_carbonate\outputs\r35_model.qppm', 'wb') as file:
    pickle.dump(r35_model, file)

In [ ]:
import matplotlib.pyplot as plt
from quick_pp.core_calibration import perm_transform
from quick_pp.rock_type import calc_r35_perm

well_data = all_data[all_data.WELL_NAME == 'HW-29'].copy()

params = {
    1: (6800, 1.9),
    2: (1400, 2.0),
    3: (110, 2.0),
    4: (17, 2.0)
}

well_data['perm_a'] = well_data['ROCK_FLAG'].map(params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
well_data['perm_b'] = well_data['ROCK_FLAG'].map(params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_trans = perm_transform(well_data['PHIT'], well_data['perm_a'], well_data['perm_b'])
perm_r35 = calc_r35_perm(well_data['R35'], well_data['PHIT'])

rock_flag_ml = rt_model.predict(well_data[['GR', 'NPHI', 'RHOB']])
perm_a_ml = pd.Series(rock_flag_ml).map(params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
perm_b_ml = pd.Series(rock_flag_ml).map(params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_ml = perm_transform(well_data['PHIT'], perm_a_ml, perm_b_ml)

r35_ml = r35_model.predict(well_data[['GR', 'NPHI', 'RHOB']])
perm_r35_ml = calc_r35_perm(r35_ml, well_data['PHIT'])

plt.figure(figsize=(15, 2))
plt.plot(well_data.DEPTH, perm_trans, label='Perm Transform')
plt.plot(well_data.DEPTH, perm_r35, label='Perm R35')
plt.plot(well_data.DEPTH, perm_ml, label='Perm ML')
plt.plot(well_data.DEPTH, perm_r35_ml, label='Perm R35 ML')
plt.scatter(well_data.DEPTH, well_data.CPERM, label='Core Perm', marker='.', c='black')
plt.yscale('log')
plt.legend()

## Predict ROCK_FLAG, R35 and PERM

The plotted predicted permeability seems aligned with the core permeability.

In [ ]:
from quick_pp.plotter import plotly_log

# Plot individual results
well_data['PERM'] = perm_r35_ml
fig = plotly_log(well_data, '')
fig.show(config=dict(scrollZoom=True))

In [ ]:
# # Save the well data
# project.update_data(well_data)
# project.save()